In [9]:
import glob
import functools
import urllib2

import pandas as pd
import json
import gspread
import pandas as pd
import os
from IPython.core.display import HTML
from oauth2client.client import SignedJwtAssertionCredentials
import numpy as np
import shutil

In [10]:
json_key = json.load(open("/home/gpratt/ipython_notebook/public clip-588adbc137f3.json"))
scope = ['https://spreadsheets.google.com/feeds']

credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'], scope)
gc = gspread.authorize(credentials)

sht1 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1ZU2mQh54jentqvhR_oMnviLGWR8Nw_x338gULzKjNDI/edit#gid=0")
ws = sht1.worksheet("Sheet1")
list_of_lists = ws.get_all_values()
manifest = pd.DataFrame(list_of_lists[1:], columns=list_of_lists[0])
manifest.is_encode = manifest.is_encode == "TRUE"
manifest.is_4000 = manifest.is_4000 == "TRUE"

manifest.method_Paper_flag = manifest.Method_Paper_flag == "TRUE" 
manifest['exp_id'] = manifest.ENCODE_ID.apply(lambda x: x.split("_")[0])

In [11]:
sht1 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1ZU2mQh54jentqvhR_oMnviLGWR8Nw_x338gULzKjNDI/edit#gid=0")


BadStatusLine: ''

In [3]:
master_processing_table = pd.read_table("/home/elvannostrand/data/ENCODE/project_stuff/ENCODE_MASTER_ID_LIST_20160301_AllDatasets.txt", 
                                        sep="\t",
                                        skiprows=1,
                                        names=["UID", "RBP_gID", "CellLine", "RBP_ENSG", "Antibody", "Lot", "CLIP_ENCODEAccID", "CLIP_Rep1ENC", "CLIP_Rep2ENC", "CLIP_InputENC", "RNASEQ_ENCODEAccID",  "Duplicate_RNASEQ_ENCODEAccID", "RNASEQ_ControlENC", "RNASEQ_KDRep1Bam", "RNASEQ_KDRep2Bam", "RNASEQ_ControlRep1Bam", "RNASEQ_ControlRep2Bam", "bar"]
)

In [7]:
rnaseq_list = pd.read_csv("/projects/ps-yeolab2/encode/analysis/rnaseq_bams_v2/20160325_rnai_manifest.csv", index_col=0)

In [8]:
rnaseq_list

,cell_type,ctrl_1,ctrl_2,ctrl_accession,kd_1,kd_2,kd_accession,rbp
0,HepG2,ENCFF350VVR,ENCFF829TUN,ENCSR042QTH,ENCFF450QLW,ENCFF584LUS,ENCSR648BSC,PUF60
1,K562,ENCFF584WAF,ENCFF674JPQ,ENCSR245BNJ,ENCFF860NWB,ENCFF288KRJ,ENCSR410ZPU,BCLAF1
2,HepG2,ENCFF350VVR,ENCFF829TUN,ENCSR042QTH,ENCFF997JOB,ENCFF904ZSQ,ENCSR459EMR,TUFM
3,K562,ENCFF092XAP,ENCFF265ZZB,ENCSR344XID,ENCFF280ZIB,ENCFF708KBX,ENCSR256PLH,QKI
4,HepG2,ENCFF561UZL,ENCFF628UPI,ENCSR689PHN,ENCFF500VFR,ENCFF962VUU,ENCSR771QMJ,GEMIN5
5,K562,ENCFF490QNF,ENCFF669VSB,ENCSR164MUK,ENCFF105GIS,ENCFF800MGC,ENCSR535YPK,SFPQ
6,HepG2,ENCFF857QSU,ENCFF052HTH,ENCSR237YZT,ENCFF878ELZ,ENCFF387CQS,ENCSR492UFS,MATR3
7,HepG2,ENCFF326XKY,ENCFF499UUZ,ENCSR491FOC,ENCFF121FVJ,ENCFF216RVH,ENCSR529QEZ,PRPF6
8,HepG2,ENCFF435FCA,ENCFF222QRV,ENCSR376RJN,ENCFF974DGK,ENCFF642RQE,ENCSR755KOM,FUBP3
9,K562,ENCFF002BTC,ENCFF002BTD,ENCSR771ZZL,ENCFF002BTB,ENCFF002BTA,ENCSR481YXD,IGF2BP3


In [4]:
master_processing_table

,UID,RBP_gID,CellLine,RBP_ENSG,Antibody,Lot,CLIP_ENCODEAccID,CLIP_Rep1ENC,CLIP_Rep2ENC,CLIP_InputENC,RNASEQ_ENCODEAccID,Duplicate_RNASEQ_ENCODEAccID,RNASEQ_ControlENC,RNASEQ_KDRep1Bam,RNASEQ_KDRep2Bam,RNASEQ_ControlRep1Bam,RNASEQ_ControlRep2Bam,bar
0,200,SLBP,NaN,ENSG00000163950.8,NaN,NaN,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,x,x,x,x,x,x,NaN
1,201,SLBP,NaN,ENSG00000163950.8,NaN,NaN,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,x,x,x,x,x,x,NaN
2,202,PTBP1,HEPG2,ENSG00000011304.12,RN011P,001,ENCSR613RQR,ENCLB484TII,ENCLB955BID,MISSING_ACCESSION,ENCSR064DXG,ENCSR064DXG,ENCSR603TCV,ENCFF155FIK.bam,ENCFF527FPF.bam,ENCFF571ARP.bam,ENCFF755GKJ.bam,NaN
3,203,HNRNPC,HEPG2,ENSG00000092199.13,RN052PW,001,ENCSR550DVK,ENCLB094PVE,ENCLB392RHM,ENCLB104YKZ,ENCSR052IYH,ENCSR052IYH,ENCSR305XWT,ENCFF031QZG.bam,ENCFF531AUK.bam,ENCFF460AAE.bam,ENCFF058QDT.bam,NaN
4,204,RBFOX2,HEPG2,ENSG00000100320.18,A300-864A,002,ENCSR987FTF,ENCLB180GIG,ENCLB696TLV,ENCLB762SQK,ENCSR767LLP,x,x,x,x,x,x,NaN
5,205,IGF2BP1,HEPG2,ENSG00000159217.5,RN007P,003,ENCSR744GEU,ENCLB134DWM,ENCLB593RRJ,ENCLB940UVB,ENCSR708GKW,ENCSR708GKW,ENCSR491FOC,ENCFF148JEK.bam,ENCFF100NIE.bam,ENCFF499UUZ.bam,ENCFF326XKY.bam,NaN
6,206,HNRNPK,HEPG2,ENSG00000165119.14,RN019P,001,ENCSR828ZID,ENCLB335FPG,ENCLB276ADD,ENCLB068HLW,ENCSR853ZJS,ENCSR853ZJS,ENCSR237YZT,ENCFF589WIS.bam,ENCFF033EZX.bam,ENCFF052HTH.bam,ENCFF857QSU.bam,NaN
7,207,MAGOH,HEPG2,ENSG00000162385.6,NaN,NaN,ENCSR158MTI,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR746EKS,ENCSR746EKS,ENCSR237YZT,ENCFF809RUA.bam,ENCFF629OTS.bam,ENCFF052HTH.bam,ENCFF857QSU.bam,NaN
8,208,IGF2BP3,HEPG2,ENSG00000136231.9,NaN,NaN,ENCSR993OLA,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR710NWE,ENCSR710NWE,ENCSR237YZT,ENCFF536RKP.bam,ENCFF930KJO.bam,ENCFF052HTH.bam,ENCFF857QSU.bam,NaN
9,209,SRSF7,HEPG2,ENSG00000115875.14,RN079PW,001,ENCSR513NDD,ENCLB131LAS,ENCLB499JKH,ENCLB994TSW,ENCSR017PRS,ENCSR017PRS,ENCSR603TCV,ENCFF479JRW.bam,ENCFF624XXX.bam,ENCFF571ARP.bam,ENCFF755GKJ.bam,NaN
